# Import Libraries

In [ ]:
#Python libraries
#Classic,data manipulation 
import numpy as np
import pandas as pd
# Plots
import plotly.graph_objects as go
import plotly.offline as po
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import plotly.express as px
import random
import plotly.figure_factory as ff
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import iplot
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
# Dataprep
from dataprep.eda import create_report
from dataprep.eda import plot_missing
from dataprep.eda import plot_correlation
from dataprep.eda import plot
# Modeling 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTEENN #resampling
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score


# Load Dataset 

In [ ]:
df=pd.read_csv("diabetes_012_health_indicators_BRFSS2015.csv") 
df.head()

# Explore Dataset

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.sample(10)

In [ ]:
df.columns

In [ ]:
# to rename the some columns important of dataset
df.rename(columns={'Diabetes_012': 'Diabetes_Type','HighBP': 'High blood pressure','HighChol':'High cholesterol','GenHlth':'General Health','DiffWalk':'Difficulty Walking'}, inplace=True)
df.head()

# Cleaning Dataset

In [ ]:
df.isna().sum() # No missing value

#### Display missing values through a library Dataprep.Eda

In [ ]:
#plot_missing(df) 

# Visualze the correlation 

In [ ]:
df.corr()

In [ ]:
corr = df.corr()
fig, ax = plt.subplots(figsize=(25,15)) 
sns.heatmap(corr,annot=True, cmap = "RdBu", linewidth = 0.30)
plt.title("Correlation matrix of features")
plt.show()

In [ ]:
df.Diabetes_Type.unique()

In [ ]:
Diabetes=df['Diabetes_Type']
Diabetes.value_counts()

In [ ]:
df['Diabetes_Type'].replace({2.0: 1.0},inplace = True)

In [ ]:
Diabetes=df['Diabetes_Type']
Diabetes.value_counts()

In [ ]:
#fig = px.pie(df['Diabetes_Type'].value_counts().reset_index(), values='Diabetes_Type', names=['No diabetes ', 'Diabetes'])
#fig.update_traces(textposition='outside', textinfo='percent+label', )
#fig.show()

In [ ]:
df['Diabetes']=df['Diabetes_Type']

In [ ]:
df['Diabetes'] = df['Diabetes_Type'].map({0.0:'No Diabetes', 1.0:'Diabetes'})

In [ ]:
df.head()

In [ ]:
#px.histogram(df, x=df['BMI'], title="The relationship between BMI and Diabetes", color="Diabetes")

In [ ]:
#px.histogram(df, x=df['HeartDiseaseorAttack'], title="Heart Diseaseor Attack vs. Diabetes", color="Diabetes")

In [ ]:
sns.set(style="darkgrid")
sns.kdeplot(data=df['Age'], shade=True)

In [ ]:
#create_report(df)

In [ ]:
diabetes_bp = df.groupby(['Diabetes_Type', 'High blood pressure']).size().reset_index(name = 'Count')
print(diabetes_bp)

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(x = 'Diabetes_Type', y = 'Count', hue = 'High blood pressure', data = diabetes_bp, palette = 'Set1')
plt.title("Dibaetes Status ~ BP")
plt.show()

In [ ]:
df['Diabetes_Type'] = df['Diabetes_Type'].astype('int')

## Experiment 1: Split data to training,validation and test set

In [ ]:
df_train_val, df_test = train_test_split(df, test_size=0.20, random_state=0)
df_train, df_val = train_test_split(df_train_val, test_size=0.20, random_state=0)

In [ ]:
# The figure above display the correlation between the features and the target, for this i choose only these features
x_train,y_train = df_train.drop(['Diabetes_Type','Diabetes'],axis=1),df_train['Diabetes_Type']
x_val,y_val = df_val.drop(['Diabetes_Type','Diabetes'],axis=1),df_val['Diabetes_Type']
x_test,y_test = df_test.drop(['Diabetes_Type','Diabetes'],axis=1),df_test['Diabetes_Type']

In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
# Train on training set, and Test on testing set
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
print("The score for kNN is")
print("Training set: {:6.2f}%".format(100*knn.score(x_train, y_train)))
print("Validation set: {:6.2f}%".format(100*knn.score(x_val, y_val)))
print("Test set: {:6.2f}%".format(100*knn.score(x_test, y_test)))

In [ ]:
y_pred = knn.predict(x_val)
print(metrics.accuracy_score(y_val, y_pred))

In [ ]:
#test the baseline model
test_pred=knn.predict(x_test)
print("\nKNN Accuracy for test set=",accuracy_score(y_test, test_pred))
print("KNN F1 score for test set=",f1_score(y_test, test_pred))

### Experiment 2: Knn by using cross vaidation

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
print(scores)

In [ ]:
# use average accuracy as an estimate of out-of-sample accuracy
print(scores.mean())

In [ ]:
# search for an optimal value of K for KNN
k_range = list(range(1, 31))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())
print(k_scores)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the value of K for KNN (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')